In [1]:
import pandas as pd



In [2]:
# Load the data into Pandas DataFrames
basics_df = pd.read_csv('/Users/lidiv/project03/title.basics.tsv.gz', sep='\t', low_memory=False)
ratings_df = pd.read_csv('/Users/lidiv/project03/title.ratings.tsv.gz', sep='\t', low_memory=False)
akas_df = pd.read_csv('/Users/lidiv/project03/title.akas.tsv.gz', sep='\t', low_memory=False)



In [3]:
print(akas_df.columns)
print(basics_df.columns)
print(ratings_df)

Index(['titleId', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'isOriginalTitle'],
      dtype='object')
Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')
            tconst  averageRating  numVotes
0        tt0000001            5.7      1993
1        tt0000002            5.8       268
2        tt0000003            6.5      1879
3        tt0000004            5.5       177
4        tt0000005            6.2      2663
...            ...            ...       ...
1351315  tt9916730            8.3        10
1351316  tt9916766            7.0        22
1351317  tt9916778            7.2        36
1351318  tt9916840            8.8         6
1351319  tt9916880            8.2         6

[1351320 rows x 3 columns]


In [4]:
#Merge the two DataFrames on 'titleId' (akas_df) and 'tconst' (basics_df)
merged_df = akas_df.merge(basics_df, left_on='titleId', right_on='tconst', how='inner')


In [5]:
# Filter for movies released in the United States
merged_df = merged_df[merged_df['region'] == 'US']


In [6]:
# Convert the 'startYear' column to numeric, handling '\\N' as NaN
merged_df['startYear'] = pd.to_numeric(merged_df['startYear'], errors='coerce')


In [7]:
# Filter for movies released between 2000 and 2021 (inclusive)
us_movies_df = merged_df[(merged_df['startYear'] >= 2000) & (merged_df['startYear'] <= 2021)]

In [9]:
# Reset the index of the 'us_movies_df' DataFrame
us_movies_df.reset_index(drop=True, inplace=True)

In [10]:
# Include only full-length movies
us_movies_df = us_movies_df.loc[basics_df['titleType'] == 'movie']



In [11]:
# Exclude movies with missing values for genre or runtime
us_movies_df = us_movies_df.dropna(subset=['genres', 'runtimeMinutes'])



In [12]:
# Exclude movies from the Documentary genre
us_movies_df = us_movies_df[~us_movies_df['genres'].str.contains('Documentary')]



In [13]:
us_movies_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0069049,3,The Other Side of the Wind,US,\N,imdbDisplay,\N,0,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,\N,122,Drama
144,tt0121803,3,Letting Go,US,\N,imdbDisplay,\N,0,tt0121803,tvMovie,Letting Go,Letting Go,0,2004.0,\N,\N,"Action,Drama"
498,tt0162030,1,Unseen,US,\N,\N,cable TV title,0,tt0162030,movie,Unseen Evil,Unseen Evil,0,2001.0,\N,93,"Horror,Sci-Fi,Thriller"
570,tt0165800,8,Brutal Truth,US,\N,dvd,\N,0,tt0165800,movie,The Giving Tree,The Giving Tree,0,2000.0,\N,89,"Drama,Thriller"
587,tt0166192,4,Finding Kelly,US,\N,imdbDisplay,\N,0,tt0166192,movie,Finding Kelly,Finding Kelly,0,2000.0,\N,88,"Comedy,Family,Mystery"


In [14]:

# Save the filtered DataFrame as a gzip-compressed CSV file
us_movies_df.to_csv('us_movies_filtered.csv.gz', compression='gzip', index=False)
